In [203]:
import numpy as np
from math import *
import matplotlib.pyplot as plt

In [245]:
class Person:
    def __init__(self, id, name, value):
        self.id = id
        self.name = name
        self.value = value
    
    def __str__(self):
        return str(self.__dict__) 
    
class Organization:
    def __init__(self, id, persons):
        self.id = id
        self.persons = persons
        
    def __str__(self):
        return 'id:' + str(self.id) + ' persons:' + ', '.join([str(x) for x in self.persons]) 
        
p1 = Person(0,'Alice', 12)
p2 = Person(1,'Bob', 23)
p3 = Person(2,'Charles', 45)

o1 = Organization(0, [p1, p2, p3])

persons = [p1, p2, p3]
print(o1)

id:0 persons:{'id': 0, 'name': 'Alice', 'value': 12}, {'id': 1, 'name': 'Bob', 'value': 23}, {'id': 2, 'name': 'Charles', 'value': 45}


In [246]:
persons[0].value = 788
print(o1)

id:0 persons:{'id': 0, 'name': 'Alice', 'value': 788}, {'id': 1, 'name': 'Bob', 'value': 23}, {'id': 2, 'name': 'Charles', 'value': 45}


In [204]:
import pygmsh
import gmsh

gmsh.initialize()

rect_width, rect_length  = 3.0, 10.0
resolution = 0.1

geom = pygmsh.geo.Geometry()


circle = geom.add_circle(
    [5,1.5,0],
    radius=0.5,
    mesh_size=resolution*0.5,
    make_surface=False
)


rect = geom.add_polygon(
    [
        [0.0, 0.0                ,0],
        [0.0, rect_width         ,0],
        [rect_length, rect_width ,0],
        [rect_length, 0.0        ,0],
    ],
    mesh_size=resolution , holes = [circle]
    
)

mesh = geom.generate_mesh(dim=2)

mesh.write("out.vtk")

geom.__exit__()

In [205]:
class Node:
    def __init__(self, id, x, y):
        self.id = id
        self.x, self.y = x, y
        self.fx, self.fy = 0,0
        self.rx, self.ry = 0,0
        self.dx, self.dy = None, None
    
    @property
    def dfix(self):
        if self.dx == 0 and self.dy == 0:
            return True
        else:
            return False
        
    @property
    def externalForce(self):
        if self.fx != 0 or self.fy != 0:
            return True
        else:
            return False
        
    def __eq__(self, obj):
        if (self.x == obj.x) and (self.y == obj.y):
            return True
        else:
            return False       
        
    def __str__(self):
        return str(self.__dict__)

class Element:
    
    maxColorVal = -9.9e19
    minColorVal = 9.9e19
    colorFunc = lambda x: x
    
    def __init__(self, id, nodes):
        self.id = id
        self.nodes = self.orderCounterClock(nodes)
        self.stress = None
        self.strain = None
        self.colorVal = 0
    
    @property
    def getmaxColorVal(self):
        return Element.maxColorVal
    
    @property
    def getminColorVal(self):
        return Element.minColorVal
    
    @property
    def getcolorFunc(self):
        return Element.colorFunc
    
    def getColor(self):
        
        try: x_ = float(self.colorVal - Element.minColorVal)/(Element.maxColorVal - Element.minColorVal)
        except ZeroDivisionError: x_ = 0.5 # cmax == cmin
        
        x = Element.colorFunc(x_)
        
        blue  = int(255* min((max((4*(0.75-x), 0.)), 1.)))
        red   = int(255* min((max((4*(x-0.25), 0.)), 1.)))
        green = int(255* min((max((4*fabs(x-0.5)-1., 0.)), 1.)))
        return (red, green, blue)
    
    def orderCounterClock(self, nodes):
        p1,p2,p3 = nodes[0], nodes[1], nodes[2]
        val = (p2.y - p1.y) * (p3.x - p2.x) - (p2.x - p1.x) * (p3.y - p2.y)
        nodes_ = nodes.copy()
        if val > 0:
            nodes[1] = nodes_[0]
            nodes[0] = nodes_[1]   
        
        assembly = []
        for n in nodes:
            assembly.append(int(n.id*2))
            assembly.append(int(n.id*2) +1)
        self.assembly = assembly
        
        return nodes
    
    def genGatherMatrix(self, nNodes):
        L = np.zeros((6,nNodes*2))
        
        for i,node in enumerate(self.nodes):
            ix, iy = int(i*2), int(i*2 +1)
            jx, jy = int(node.id*2), int(node.id*2 +1)
            L[ix, jx] = 1
            L[iy, jy] = 1
            
        self.L = L
        return L
    
    def __str__(self):
        return str(self.id) + ': [ ' + ', '.join([str(node.id) for node in self.nodes]) + ' ]'
        

def setNodeAttribute(id, nodes, attribute, val):
    for n in nodes:
        if n.id == id:
            n.__dict__[attribute] = val

In [206]:
maxNode = 0
for cell in mesh.cells[1].data:
    for node in cell:
        if node > maxNode:
            maxNode = node
print(maxNode)

meshCells = mesh.cells[1].data - np.full(np.shape(mesh.cells[1].data), 1, dtype=np.uint64)
meshPoints = mesh.points[1:]

4598


In [208]:
elements = []
allNodes = []
nE = 0
for nodesId in meshCells:
    points = [meshPoints[i] for i in nodesId]
    newElement = Element(id=nE, nodes=[Node(i, point[0], point[1]) for i, point in zip(nodesId,points)])
    elements.append(newElement)
    for node in newElement.nodes:
        if not node in allNodes:
            allNodes.append(node)
    nE += 1
  
allNodes = sorted(allNodes, key= lambda x: x.id)    

In [209]:
coord_matrix = []
for i,node in enumerate(allNodes):
    coord_matrix.append(node.x)
    coord_matrix.append(node.y)

In [210]:
v = 0.1
E = 200.0e9

D = (E/(1-v**2)) * np.array([
    [1, v, 0],
    [v, 1, 0],
    [0, 0, (1-v)/2],
])

In [211]:
Ne = len(elements)
Nnodes = len(allNodes)

L_gather = [element.genGatherMatrix(Nnodes) for element in elements]

coord_matrix = []
for i,node in enumerate(allNodes):
    coord_matrix.append(node.x)
    coord_matrix.append(node.y)

f = np.zeros((int(2*Nnodes), 1))
d = np.full((int(2*Nnodes), 1), None)
r = np.full((int(2*Nnodes), 1), None)

for i in range(0,len(coord_matrix) ,2):  
    
    if coord_matrix[i] == rect_length:
        f[i+1] = -1.0e3
        setNodeAttribute(id=int(i/2), elements=elements, attribute='externalForce', val=True)
        
    if coord_matrix[i] == 0:
        d[i] = 0
        d[i+1] = 0
        
        setNodeAttribute(id=int(i/2), elements=elements, attribute='dfix', val=True) 
        
    else:
        r[i] = 0
        r[i+1] = 0
    
        

In [212]:
import drawMesh
import importlib

importlib.reload(drawMesh)

render = drawMesh.MeshRender()
render.legend = False
render.autoScale = True
render.colorElements = False

render.drawElements(elements)

In [213]:
def Ae(Pe):
    x1,y1 = Pe[0], Pe[1]
    x2,y2 = Pe[2], Pe[3]
    x3,y3 = Pe[4], Pe[5]
    result = 0.5*((x2*y3 - x3*y2)-(x1*y3- x3*y1)+(x1*y2-x2*y1))
    if result == 0:
        result = 1e-20
    return result
    

In [214]:
def Be(ie):
    Pe = np.matmul(L_gather[ie],coord_matrix)
    x1,y1, x2,y2, x3,y3 = Pe[0], Pe[1], Pe[2], Pe[3], Pe[4], Pe[5]
    return (0.5/Ae(Pe)) * np.array([
        [(y2-y3) ,  0    , (y3-y1),  0   ,   (y1-y2),   0   ],
        [   0    , (x3-x2),  0    , (x1-x3),     0   ,(x2-x1)],
        [(x3-x2) , (y2-y3), (x1-x3), (y3-y1), (x2-x1) ,(y1-y2)],
    ])
    

$
\begin{equation}
K^e = A^e (B^e)^T D B^e
\end{equation}
$

$
\begin{equation}
K = \sum (L^e)^T K^e L^e
\end{equation}
$

In [215]:
import time

In [216]:
import sympy as sp

k00, k01, k02, k10, k11, k12, k20, k21, k22 = sp.symbols(r'k_{00}, k_{01}, k_{02}, k_{10}, k_{11}, k_{12}, k_{20}, k_{21}, k_{22}')

Ke = sp.Matrix([
    [k00, k01, k02],
    [k10, k11, k12],
    [k20, k21, k22]]
)

'''
Ke = sp.MatrixSymbol('k', 3, 3)
'''

Le = sp.Matrix([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0]]
)

Le.T * Ke * Le

Matrix([
[0,      0,      0,      0],
[0, k_{22}, k_{21}, k_{20}],
[0, k_{12}, k_{11}, k_{10}],
[0, k_{02}, k_{01}, k_{00}]])

In [217]:
def assemblyK(K, Ke, nodeAssembly):
    #K_ = np.zeros((Nnodes*2,Nnodes*2))
    for i,t in enumerate(nodeAssembly):
        for j,s in enumerate(nodeAssembly):
            K[t][s] += Ke[i][j]

In [218]:
ie = 0
Pe = np.matmul(L_gather[ie],coord_matrix)   
Bie = Be(ie)  
Ke = Ae(Pe)* np.matmul(Bie.T, np.matmul(D, Bie))
e = elements[ie]


In [219]:
K = np.zeros((Nnodes*2,Nnodes*2))
B_list = []

#print('Pe', 'Bie', 'Ke', 'K', sep='\t')
for ie in range(Ne):
    
    e = elements[ie]
    
    #t0 = time.time()
    
    Pe = np.matmul(L_gather[ie],coord_matrix)
    
    #t1 = time.time()
    
    Bie = Be(ie)
    
    #t2 = time.time()
    
    Ke = Ae(Pe)* np.matmul(Bie.T, np.matmul(D, Bie))
    
    #t3 = time.time()
    
    nodeAssembly = e.assembly
    
    assemblyK(K, Ke, nodeAssembly)
    
    #t4 = time.time()
    
    B_list.append(Bie)
    
    #print(round(1.0e4*(t1-t0),1), round(1.0e4*(t2-t1),1), round(1.0e4*(t3-t2),1), round(1.0e4*(t4-t3),1), sep='\t',  end='\r')
    
    
    if ie%20 == 0:
        print(round(ie/Ne*100,2),'%', end='\r')

print(round(100.00,2),'%', end='\r')

$\begin{equation}
Kd = f + r
\end{equation}$

In [220]:
rowsrk = [i for i in range(len(d)) if d[i] == None]
rowsdk = [i for i in range(len(r)) if r[i] == None]

In [221]:
KB = np.zeros((len(rowsrk),len(rowsrk)))
KA = np.zeros((len(rowsdk),len(rowsrk)))

fk = np.array([r[i] for i in rowsrk]) + np.array([f[i] for i in rowsrk])
dk = np.array([d[i] for i in rowsdk]) 

K_rows, K_cols = np.shape(K)

for i in range(np.shape(KB)[0]):
    for j in range(np.shape(KB)[1]):
        KB[i][j] = K [rowsrk[i]][rowsrk[j]]

for i in range(np.shape(KA)[0]):
    for j in range(np.shape(KA)[1]):
        KA[i][j] = K [rowsdk[i]][rowsrk[j]]

In [222]:
du = np.matmul(np.linalg.inv(KB), fk)
fu = np.matmul(KA,du)

In [223]:
d_total = d.copy()

for i, d_solve in zip(rowsrk, du):
    d_total[i] = d_solve

dx, dy = [],[]
for i in range(0,len(d_total),2):
    dx.append(d_total[i])
    dy.append(d_total[i + 1])

In [224]:
def calculateVonMises(sx, sy, sxy):
    return sqrt(sx**2 + sy**2 + 3*(sxy**2) - sx*sy)

In [227]:
deformation_scale = 1.0e4

maxd, mind = max(d_total)[0], min(d_total)[0]

def rgb(mag, cmin, cmax):
    """ Return a tuple of floats between 0 and 1 for R, G, and B. """
    # Normalize to 0-1
    try: x = float(mag-cmin)/(cmax-cmin)
    except ZeroDivisionError: x = 0.5 # cmax == cmin
    
    blue  = int(255* min((max((4*(0.75-x), 0.)), 1.)))
    red   = int(255* min((max((4*(x-0.25), 0.)), 1.)))
    green = int(255* min((max((4*fabs(x-0.5)-1., 0.)), 1.)))
    return (red, green, blue)

average = lambda x: (sum(x)/len(x))[0]

Element.colorFunc = lambda x: exp(-x)

for i,element in enumerate(elements):
    Le = element.L
    de = np.matmul(Le, d_total)
    
    strain_e = np.matmul(B_list[i],de)
    stress_e = np.matmul(D, strain_e)

    dx_avg = average([de[0], de[2], de[4]])
    dy_avg = average([de[1], de[3], de[5]])
    
    element.strain = strain_e
    element.stress = stress_e
    
    element.colorVal = calculateVonMises(element.stress[0][0], element.stress[1][0], element.stress[2][0])
    
    if element.colorVal > Element.maxColorVal:
        Element.maxColorVal = element.colorVal
    if element.colorVal < Element.minColorVal:
        Element.minColorVal = element.colorVal
        
    for i,n in enumerate(element.nodes):
        ix, iy = int(i*2), int(i*2) + 1
        n.x += de[ix]*deformation_scale
        n.y += de[iy]*deformation_scale
    

In [228]:
import drawMesh
import importlib

importlib.reload(drawMesh)

render = drawMesh.MeshRender()
render.legend = True
render.autoScale = True
render.legendDiscretize = 10
render.legendTitle = 'von-mises (Pa)'
render.drawElements(elements)